In [1]:
# 這是準備訓練資料的程式碼，主要是將資料集的圖片檔案，以及對應的標籤檔案，移在一起，並且將標籤檔案轉換成數字的索引，以便後續的訓練使用
# source C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\...
# target C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\train\
# label_target C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\train\label.txt
# separator label.txt to train.txt and test.txt

# include check image orentation
# include check image size and padding to width
# include encode train.txt and test.txt label to index

In [9]:
import glob
import os 
import cv2
import shutil
import time
from PIL import Image, ImageOps
import tqdm
# from tqdm.notebook import tqdm


In [10]:
# generate folder list
# folders = [f for f in os.listdir('C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\')]
folders = [f for f in os.listdir('C:/Develop/CRNN_Chinese_Characters_Rec_data/datasets/')]
# folders 裡面只有 train 資料夾(不要 vaild test)跟其他實際資料的各個資料夾
# 因為 train/labels.txt 檔案為附加寫入，所以要刪掉讓他重新產生新檔案
folders.remove('train')

# folders.remove('opposite_01')
# folders = ['enum_02_20k','vrt_06_50k']

print(len(folders),folders)

45 ['ibodict_c2w_cs0_na2_wd1153_b0_al1_rbl_m2_tc_ft', 'ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0', 'ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0_rk', 'ibodict_c2w_cs2_na2_wd1153_b1_al2_rbl_m1', 'lcn_c2w_w20_cs0_sw1_na2_wd1153_b1_al2_rbl_m0_rk', 'lcn_c2w_w30_cs1_sw0_na2_wd1153_b1_al1_rbl_m0_rk', 'lcn_c2w_w40_cs0_sw0_na2_wd1153_b1_al2_rbl_m0_do', 'lcn_c3w_w30_cs1_sw0_na2_wd1153_b0_al1_rbl_m1', 'lcn_c3w_w30_cs1_sw1_na2_wd1153_b0_al0_rbl_fi', 'let_c1w_w7_cs1_sw1_na2_wd1153_b1_al2_rbl_fi', 'let_c5k_w7_cs1_sw1_na2_wd1153_b0_al0_rbl_m1_do', 'let_c5k_w7_cs1_sw1_na2_wd1153_b0_al1_rbl_m1_rk', 'let_c5k_w7_cs1_sw1_na2_wd1153_b1_al0_rbl_fi_tc', 'let_num_c1w_w6_cs1_sw1_na2_wd1153_b3_al2_rbl_m0_tc', 'let_num_c1w_w7_cs1_sw1_na2_wd1153_b3_al1_rbl_fi_tc', 'let_num_c2w_w7_cs1_sw1_na2_wd1153_b1_al0_rbl_fi', 'let_num_c5k_w6_cs1_sw1_na2_wd1153_b1_al2_rbl_m0_tc', 'let_num_c5k_w7_cs1_sw1_na2_wd1153_b1_al2_rbl_m0_ft', 'let_num_c5k_w9_cs1_sw1_na2_wd1153_b1_al1_rbl_fi_ft', 'lzh_c2w_w20_cs0_sw1_na2_wd1153_b1_al2_rbl_m

In [11]:
import time
import threading
import queue

def proc_labels_txt(folder):
    print('process {} folder labels txt'.format(folder))
    # 根據檔案清單，合併到 train/labels.txt中，但是圖片原處處理不搬移，因為檔案太多讀不動
    nlines = []
    with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\labels.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
#         for line in tqdm.tqdm(lines):
        for line in (lines):
            image_path = line.split(' ')[0]
            try:
                ### 這裡做旋轉
                img = cv2.imread('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path)
                h,w,c = img.shape
                if h > w: # rotate 直的轉橫的逆時針90度
                    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                    cv2.imwrite('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path, img)
                location = len(image_path)
                nlines.append(folder + '/' + image_path + line[location:])
            except:
#                 print(folder + '/' + image_path + line[location:])
                pass
    # 因為同時一筆一筆寫入 train/labels.txt 會出錯，所以拆開，資料夾分開最後再依序寫入
    with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\nlabels.txt', 'w', encoding='utf-8') as f:
        for item in nlines:
            f.write("{}".format(item))      
    print('process {} folder labels txt done.'.format(folder))


import threading
import time

# 子執行緒類別
class MyThread(threading.Thread):
  def __init__(self, num, folder):
    threading.Thread.__init__(self)
    self.num = num
    self.folder = folder

  def run(self):
    print("Thread", self.num, self.folder)
    proc_labels_txt(self.folder)
    time.sleep(0.1)

# 建立 i 個子執行緒
threads = []
for i, folder in enumerate(folders):
  threads.append(MyThread(i, folder))
  threads[i].start()

# 主執行緒繼續執行自己的工作
# ...

# 等待所有子執行緒結束
for i in range(len(folders)):
  threads[i].join()

# write a empyt txt file
f = open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\labels.txt', 'w', encoding='utf-8')
f.close()

for folder in folders:
    f1 = open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\labels.txt', 'a+', encoding='utf-8')
    f2 = open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\nlabels.txt', 'r', encoding='utf-8')
    f1.write(f2.read()) 
    f1.close()
    f2.close()

print("all process Done.")

ThreadThread 1 ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0
process ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0 folder labels txt
 0 ibodict_c2w_cs0_na2_wd1153_b0_al1_rbl_m2_tc_ft
process ibodict_c2w_cs0_na2_wd1153_b0_al1_rbl_m2_tc_ft folder labels txt
Thread 2 ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0_rk
process ibodict_c2w_cs1_na2_wd1153_b0_al0_rbl_m0_rk folder labels txt
Thread 3 ibodict_c2w_cs2_na2_wd1153_b1_al2_rbl_m1
process ibodict_c2w_cs2_na2_wd1153_b1_al2_rbl_m1 folder labels txt
Thread 4 lcn_c2w_w20_cs0_sw1_na2_wd1153_b1_al2_rbl_m0_rk
process lcn_c2w_w20_cs0_sw1_na2_wd1153_b1_al2_rbl_m0_rk folder labels txt
Thread 5 lcn_c2w_w30_cs1_sw0_na2_wd1153_b1_al1_rbl_m0_rk
process lcn_c2w_w30_cs1_sw0_na2_wd1153_b1_al1_rbl_m0_rk folder labels txt
Thread 6 lcn_c2w_w40_cs0_sw0_na2_wd1153_b1_al2_rbl_m0_do
process lcn_c2w_w40_cs0_sw0_na2_wd1153_b1_al2_rbl_m0_do folder labels txt
Thread 7 lcn_c3w_w30_cs1_sw0_na2_wd1153_b0_al1_rbl_m1
process lcn_c3w_w30_cs1_sw0_na2_wd1153_b0_al1_rbl_m1 folder labels 

In [5]:
### 這跟上面那格依樣功能(重複，別做!!!)

# for folder in folders:
#     print(folder)
#     # 根據檔案清單，合併到 train/labels.txt中，但是圖片原處處理不搬移，因為檔案太多讀不動
#     with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\labels.txt', 'r', encoding='utf-8') as f:
#         lines = f.readlines()
#         for line in tqdm.tqdm(lines):
#             image_path = line.split(' ')[0]
#             try:
#                 img = cv2.imread('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path)
#                 h,w,c = img.shape
#                 # print(img.shape)
#                 if h > w: # rotate 直的轉橫的逆時針90度
#                     img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
#                     # h,w,c = img.shape
#                     # print(img.shape)
#                     # cv2.imshow('img', img)
#                     # pause = cv2.waitKey(0)
#                     cv2.imwrite('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path, img)
#                     # print('rotate')
#                 # os.rename('C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path, 'C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\'+image_path)
#                 # copy to target folder 這樣可以重複執行(不過labels.txt會重複要先刪除)
#     #             shutil.copyfile('C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\'+folder+'\\'+image_path, 'C:\Application\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\'+image_path)
#                 # 計算檔名長度及位置，修改路徑(原位置不搬移)後，產生新的label line
#                 location = len(image_path)
#                 line = folder + '/' + image_path + line[location:]
#                 # 將標籤寫道檔案中 (train\label.txt) 模式是 append
#                 with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\labels.txt', 'a', encoding='utf-8') as f:
#                     f.write(line)
#             except:
#                 pass
#     time.sleep(0.1) # 暫停0.1秒避免 labels.txt 檔案被占用


In [6]:
# f.close()
# del lines
# import gc
# gc.collect(generation=2)

In [12]:
# 因為產生直的圖片尺寸不一，所以要padding到一樣的尺寸
# 710W張圖片，執行時間約110分鐘
# 图像最大尺寸大小： 1157 32  w_max, h_max
# 图像最大尺寸名稱： C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\w14_cs0sw1_fi_or\0.jpg  w_max_img_name
# 图像最大尺寸名稱： C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\bo_dictc30wcs1sw2b1\0.jpg  h_max_img_name

import glob
images = glob.glob('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\*\\*.jpg', recursive = True)
print(len(images))
w_max, h_max = 0, 0
w_max_img_name = []  # 用來存放  w_max 圖片名稱
h_max_img_name = []  # 用來存放  h_max 圖片名稱
for image in tqdm.tqdm(images):
    # 不用打開圖片，直接獲取圖片尺寸大小（width, height）
    with Image.open(image) as img:
    # 获取图像尺寸大小（width, height）
        w, h = img.size
        if w_max < w:
            w_max = w
            w_max_img_name = image
        if h_max < h:
            h_max = h
            h_max_img_name = image
    # 打印尺寸大小
print("图像最大尺寸大小：", w_max, h_max)
print("图像最大尺寸名稱：", w_max_img_name)
print("图像最大尺寸名稱：", h_max_img_name)

811944


100%|██████████| 811944/811944 [00:49<00:00, 16481.95it/s]

图像最大尺寸大小： 1153 32
图像最大尺寸名稱： C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\ibodict_c2w_cs0_na2_wd1153_b0_al1_rbl_m2_tc_ft\0.jpg
图像最大尺寸名稱： C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\ibodict_c2w_cs0_na2_wd1153_b0_al1_rbl_m2_tc_ft\0.jpg


In [7]:
import time
import threading
import queue

w_max = 1153
def proc_padding_image(image, w_max):
    with Image.open(image) as img:
        w, h = img.size
        if w < w_max:
            img = ImageOps.pad(img, (w_max, h), method=Image.LINEAR, color=(255,255,255), centering=(0, 0))
            img.save(image)
            # print('padding')    

import threading
import time

# 子執行緒類別
class MyThread(threading.Thread):
  def __init__(self, num, image, w_max):
    threading.Thread.__init__(self)
    self.num = num
    self.image = image
    self.w_max = w_max

  def run(self):
    # print("Thread", self.num, self.image, self.w_max)
    proc_padding_image(self.image, self.w_max)
    # time.sleep(0.0001)

# 建立 i 個子執行緒
threads = []
for i, image in enumerate(images):
  threads.append(MyThread(i, image, w_max))
  threads[i].start()

# 主執行緒繼續執行自己的工作
# ...

# 等待所有子執行緒結束
for i in range(len(images)):
  threads[i].join()

print("all process Done.")   # 43m58s

C:\Users\richardchiu\AppData\Local\Temp\ipykernel_5132\2127678199.py:10: DeprecationWarning: LINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use BILINEAR or Resampling.BILINEAR instead.
  img = ImageOps.pad(img, (w_max, h), method=Image.LINEAR, color=(255,255,255), centering=(0, 0))


all process Done.


DeprecationWarning: LINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use BILINEAR or Resampling.BILINEAR instead.

In [9]:
# # 這裡很慢 
# # 前面步驟已做旋轉，這裡才不會出問題
# # 因為產生直的圖片尺寸不一，所以要padding到一樣的尺寸

# # w_max, h_max = 888, 32
# w_max, h_max = 1157, 32

# # padding
# for image in tqdm.tqdm(images):
#     with Image.open(image) as img:
#         w, h = img.size
#         if w < w_max:
#             img = ImageOps.pad(img, (w_max, h), method=Image.LINEAR, color=(255,255,255), centering=(0, 0))
#             img.save(image)
#             # print('padding')


In [13]:
# 檢查一下圖片數量
with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\labels.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
print(len(lines))   # 7364498  


811944


In [14]:
# 隨機打亂標籤檔案
import random
random.shuffle(lines)
random.shuffle(lines)
random.shuffle(lines)
random.shuffle(lines)
random.shuffle(lines)
random.shuffle(lines)
total_num = len(lines)
rate = 0.9
print(len(lines), int(total_num * rate), total_num-int(total_num * rate))

with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\train.txt', 'w', encoding='utf-8') as f:
    for line in lines[:int(total_num * rate)]:
    # for line in lines[:7000000]:
        f.write(line)

with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\test.txt', 'w', encoding='utf-8') as f:
    for line in lines[int(total_num * rate):]:
    # for line in lines[7000000:-10]:    # 最後的幾筆不要，避免空白之類的怪問題
        f.write(line)

811944 730749 81195


In [15]:
'''
產生中文训练集标签 只有 train & valid (test) 兩部分，沒有 test!
'''

# chinese characters dictionary for 3.6 million data set.
with open('C:\Develop\CRNN_Chinese_Characters_Rec\lib\dataset\\txt\char_std_zhcn_8000.txt', 'r', encoding='utf-8') as file:
    char_dict = {char.replace('\n','') : num for num, char in enumerate(file.readlines())}

# # processing output
with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\train.txt', 'r', encoding='utf-8') as file:
    new_output = []
    for segment_list in file.readlines():
        segment_list = segment_list.replace('\n','')
        img_name = segment_list.split(' ')[0]
        img_name_len = len(img_name)
        # print(segment_list)
        segment_list = segment_list[img_name_len+1:]
        # print(segment_list)
        new_segment_list = ''
        for w in segment_list:
            new_segment_list += str(char_dict[w]) + ' '
        # print(new_segment_list)
        new_output.append(img_name + ' ' + new_segment_list[:-1]) # 去掉最後一個空白
    print(len(new_output))

# final output
with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\train_code.txt', 'w', encoding='utf-8') as file:
    [file.write(val+'\n') for val in new_output]

# valid part 5%
# # processing output
with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\test.txt', 'r', encoding='utf-8') as file:
    new_output = []
    for segment_list in file.readlines():
        segment_list = segment_list.replace('\n','')
        img_name = segment_list.split(' ')[0]
        img_name_len = len(img_name)
        # print(segment_list)
        segment_list = segment_list[img_name_len+1:]
        # print(segment_list)
        new_segment_list = ''
        for w in segment_list:
            new_segment_list += str(char_dict[w]) + ' ' # 每個字構面加一個空白符
        # print(new_segment_list)
        new_output.append(img_name + ' ' + new_segment_list[:-1]) # 去掉最後一個空白
    print(len(new_output))

# final output
with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\test_code.txt', 'w', encoding='utf-8') as file:
    [file.write(val+'\n') for val in new_output]

    


730749
81195


In [9]:
char_dict[' '] == 1

True

In [10]:
char_dict['妥']

1396

In [16]:
import os
import time

os.getcwd()
os.chdir('C:/develop/CRNN_Chinese_Characters_Rec')
os.getcwd()



'C:\\develop\\CRNN_Chinese_Characters_Rec'

In [16]:
os.system("python train.py")

In [ ]:
# # 特別處理   這不需要了 是誤會一場 圖片讀取速度很快不用特別放一起
# import os, shutil 
# try:
#     os.mkdir('C:/Develop/CRNN_Chinese_Characters_Rec_data/datasets/valid')
# except:
#     pass
# # # processing output
# with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\test_code.txt', 'r', encoding='utf-8') as file:
#     new_output = []
#     for segment_list in file.readlines():
#         segment_list = segment_list.replace('\n','')
#         img_name = segment_list.split(' ')[0]
#         jpg_name = img_name.split('/')[1]
#         shutil.copyfile('C:/Develop/CRNN_Chinese_Characters_Rec_data/datasets/'+img_name, 'C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\valid\\'+jpg_name)
#         img_name_len = len(img_name)
#         segment_list = segment_list[img_name_len+1:]
#         new_segment = 'valid/'+jpg_name+' '+segment_list  # a str
#         new_output.append(new_segment) # 去掉最後一個空白
#     print(len(new_output))

# # final output
# with open('C:\Develop\CRNN_Chinese_Characters_Rec_data\datasets\\train\\test_code_valid.txt', 'w', encoding='utf-8') as file:
#     [file.write(val+'\n') for val in new_output]


In [ ]:
import math

p1 = (215, 179)
p2 = (448, 177)

angle = math.degrees(math.atan2(p2[1] - p1[1], p2[0] - p1[0]))

print(angle)

-0.49179718794591293
